In [ ]:
!pip install wandb

In [86]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from tqdm import tqdm
import wandb

In [96]:
wandb.init(project="Mnist Persian")

acc,▁▃▆▇▇█▇███
epochs,▁▂▃▃▄▅▆▆▇█
loss,█▆▃▃▂▂▂▁▁▁
acc,0.76069
epochs,10
loss,1.70101


In [97]:
class Model(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3, 32, (3, 3), (1, 1), (1, 1)) 
    self.conv2 = nn.Conv2d(32, 64, (3, 3), (1, 1), (1, 1))
    self.conv3 = nn.Conv2d(64, 128, (3, 3), (1, 1), (1, 1))
    # self.conv4 = nn.Conv2d(128, 128, (3, 3), (1, 1), (1, 1))

    self.fc1 = nn.Linear(128*8*8 ,128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    conv1 = F.relu(self.conv1(x))
    conv1 = F.max_pool2d(conv1, kernel_size=(2, 2))

    conv2 = F.relu(self.conv2(conv1))
    conv2 = F.max_pool2d(conv2, kernel_size=(2, 2))

    conv3 = F.relu(self.conv3(conv2))
    conv3 = F.max_pool2d(conv3, kernel_size=(2, 2))

    # conv4 = F.relu(self.conv4(conv3))
    # conv4 = F.max_pool2d(conv4, kernel_size=(2, 2))

    flatten = torch.flatten(conv3, start_dim=1)
    flatten = torch.dropout(flatten, 0.2, train=True)

    fc1 = self.fc1(flatten)
    fc1 = torch.dropout(fc1, 0.4, train=True)

    fc2 = self.fc2(fc1)
    output = torch.softmax(fc2, dim=1)

    return output


In [98]:
# args.gpu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda" if torch.cuda.is_available() and args.gpu else "cpu")
model = Model().to(device)
wandb.watch(model)

[]

In [99]:
# Hyper Parameters

batch_size = 64
epochs = 20
config = wandb.config
config.learning_rate = 0.001

In [100]:
from torchvision.transforms.transforms import Resize
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.Resize((70, 70)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

dataset = torchvision.datasets.ImageFolder(root="/content/drive/MyDrive/MNIST_persian", transform = transform)
train_data = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [101]:
# compile

optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
loss_function = nn.CrossEntropyLoss()

In [102]:
# accuracy

def calc_acc(preds, labels):
  _, pred_max = torch.max(preds, 1)
  acc = torch.sum(pred_max == labels.data, dtype=torch.float64) / len(preds)
  return acc

In [103]:
# train

model.train()

for epoch in range(epochs):
  train_loss = 0.0
  train_acc = 0.0
  for images, labels in tqdm(train_data):

    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    preds = model(images)

    loss = loss_function(preds, labels)
    loss.backward()

    optimizer.step()

    train_loss += loss
    train_acc += calc_acc(preds, labels)

  total_loss = train_loss / len(train_data)
  total_acc = train_acc / len(train_data)
  print(f"Epoch: {epoch+1}, Loss: {total_loss}, Accuracy: {total_acc}")

  wandb.log({'epochs':  epoch + 1,
              'loss': total_loss,
              'acc': total_acc
                              })

100%|██████████| 19/19 [00:03<00:00,  5.54it/s]


Epoch: 1, Loss: 2.30965518951416, Accuracy: 0.11293859649122807


100%|██████████| 19/19 [00:03<00:00,  6.06it/s]


Epoch: 2, Loss: 2.243826150894165, Accuracy: 0.2064144736842105


100%|██████████| 19/19 [00:03<00:00,  6.06it/s]


...

Epoch: 18, Loss: 1.5514851808547974, Accuracy: 0.9150219298245614


100%|██████████| 19/19 [00:03<00:00,  6.02it/s]


Epoch: 19, Loss: 1.553490400314331, Accuracy: 0.9106359649122806


100%|██████████| 19/19 [00:03<00:00,  5.15it/s]

Epoch: 20, Loss: 1.5443037748336792, Accuracy: 0.9207785087719298


In [104]:
torch.save(model.state_dict(), "mnist_persian.pth")